In [ ]:
from google.colab import drive
import huggingface_hub
from google.colab import userdata
import pandas as pd
import json
import os
import torch

In [ ]:
drive.mount('/content/drive')
huggingface_hub.login("HF_token")

In [ ]:
'''
%cd /content/drive/MyDrive/5398
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]
'''

In [ ]:
!python -m pip install -e .
!python -m pip list | grep llamafactory
!pip install -e .[torch,bitsandbytes]
!pip install rouge
#remember to restart session

In [ ]:
import time
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from rouge import Rouge

In [ ]:
%cd /content/drive/MyDrive/5398/LLaMA-Factory

# **1. Llama3.1 8b**

**1.1 Fine Tune**

In [ ]:
args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="meta-llama/Llama-3.1-8B",
  dataset="df_train_system",
  template="llama3",
  finetuning_type="lora",
  lora_target="q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj",
  output_dir="llama3.1_lora",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=16,
  lr_scheduler_type="cosine",
  logging_steps=5,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_grad_norm=1.0,
  loraplus_lr_ratio=16.0,
  fp16=True,
  report_to="none",
)

json.dump(args, open("train_llama3.1.json", "w", encoding="utf-8"), indent=2)

%cd /content/drive/MyDrive/5398/LLaMA-Factory
!llamafactory-cli train "/content/drive/MyDrive/5398/LLaMA-Factory/train_llama3.1.json"

In [ ]:
args = dict(
  model_name_or_path="meta-llama/Llama-3.1-8B",
  adapter_name_or_path="llama3.1_lora",
  template="llama3",
  finetuning_type="lora",
  max_new_tokens = 512
)
chat_model = ChatModel(args)

In [ ]:
system_text = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ...\n\n"
messages = [
    {"role": "user", "content": system_text + "\n" +"[Company Introduction]:\n\nAmerican Express Co is a leading entity in the Financial Services sector. Incorporated and publicly traded since 1977-05-18, the company has established its reputation as one of the key players in the market. As of today, American Express Co has a market capitalization of 168338.49 in USD, with 723.87 shares outstanding.\n\nAmerican Express Co operates primarily in the US, trading under the ticker AXP on the NEW YORK STOCK EXCHANGE, INC.. As a dominant force in the Financial Services space, the company continues to innovate and drive progress within the industry.\n\nFrom 2024-02-11 to 2024-02-18, AXP's stock price increased from 211.81 to 211.90. News during this period are listed below:\n\n[Headline]: American Express: Buy, Sell, or Hold?\n[Summary]: With shares at all-time highs, investors still won't struggle in finding reasons to be bullish.\n\n[Headline]: American Express Co. stock underperforms Monday when compared to competitors\n[Summary]: Shares of American Express Co. slipped 0.10% to $212.26 Monday, on what proved to be an all-around mixed trading session for the stock market, with the Dow...\n\n[Headline]: The Next Berkshire Hathaway? 3 Blue-Chip Stocks That Investors Shouldn’t Ignore.\n[Summary]: Blue chip stocks are what many investors look for when trying to add new stocks to their portfolios. What really are these companies? Well, a blue chip company is generally a well-known and well-established company that has a history of generating consistent profits. They have grown their revenue in the past and have worked to make a successful company. That’s why these are usually more reliable to invest in since they are not small startup companies that just hit the market and carry tons of ri\n\n[Headline]: Tracking Mario Gabelli's Gabelli Funds 13F Portfolio - Q4 2023 Update\n[Summary]: Gabelli Funds' 13F portfolio value increased by approximately 5% in Q4 2023. Click here to read more about Mario Gabelli's holdings and trades for Q4 2023.\n\n[Headline]: American Express Co. stock underperforms Thursday when compared to competitors despite daily gains\n[Summary]: Shares of American Express Co. inched 0.77% higher to $212.53 Thursday, on what proved to be an all-around positive trading session for the stock market,...\n\nFrom 2024-02-18 to 2024-02-25, AXP's stock price increased from 211.90 to 213.90. News during this period are listed below:\n\n[Headline]: The Dow is due for a change. These sectors may be affected.\n[Summary]: The keepers of the Dow Jones Industrial Average will have to make at least one change to the venerable index next week, and there’s a very good chance...\n\n[Headline]: Discover Stock Jumps on Capital One Deal News\n[Summary]: Shares of Discover Financial Services jumped in early trading following the news that Capital One will buy the credit-card company for more than $35 billion.  Discover's shares were up 14% in recent trading, while Capital One's rose 1%.  Capital One stock was up more than 4%.\n\n[Headline]: Got $1,000? 3 Payments Stocks to Buy and Hold Forever\n[Summary]: Americans spend trillions of dollars on debit and credit cards; these three companies are behind almost every transaction.\n\n[Headline]: Are You a Growth Investor? This 1 Stock Could Be the Perfect Pick\n[Summary]: The Zacks Style Scores offers investors a way to easily find top-rated stocks based on their investing style. Here's why you should take advantage.\n\n[Headline]: Nearly One-Fifth of Warren Buffett-Led Berkshire Hathaway's $368 Billion Stock Portfolio Is Invested in These 2 Financial Giants\n[Summary]: These two industry heavyweights should be on your investing radar.\n\nSome recent basic financials of AXP, reported at 2023-12-31, are presented below:\n\n[Basic Financials]:\n\nassetTurnoverTTM: 0.2635\nbookValue: 28057\ncashRatio: 0.31986958884260097\ncurrentRatio: 0.7395\nebitPerShare: 3.4553\neps: 2.6589\nev: 268910.19\nfcfMargin: 0.3691\nfcfPerShareTTM: 23.5076\ngrossMargin: 0.6334\nlongtermDebtTotalAsset: 0.1833\nlongtermDebtTotalCapital: 0.232\nlongtermDebtTotalEquity: 1.706\nnetDebtToTotalCapital: 0.6415\nnetDebtToTotalEquity: 4.7185\nnetMargin: 0.1125\noperatingMargin: 0.1462\npayoutRatioTTM: 0.2126\npb: 4.8659\npeTTM: 16.3032\npfcfTTM: 7.6988\npretaxMargin: 0.1462\npsTTM: 2.0881\nquickRatio: 0.7395\nreceivablesTurnoverTTM: 1.1117\nroaTTM: 0.0338\nroeTTM: 0.3099\nroicTTM: 0.0422\nrotcTTM: 0.053\nsalesPerShare: 23.6369\nsgaToSale: 0.3666\ntotalDebtToEquity: 6.355\ntotalDebtToTotalAsset: 0.6829\ntotalDebtToTotalCapital: 0.864\ntotalRatio: 1.1204\n\nBased on all the information before 2024-02-25, let's first analyze the positive developments and potential concerns for AXP. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. Then make your prediction of the AXP cryptocurrency price movement for next week (2024-02-25 to 2024-03-03). Provide a summary analysis to support your prediction."}
]

response = ""
for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text

torch_gc()

In [ ]:
#merge and saving model

args = dict(
  model_name_or_path="meta-llama/Llama-3.1-8B",
  adapter_name_or_path="llama3.1_lora",
  template="llama3",
  finetuning_type="lora",
  export_dir="llama3.1_lora_merged",
  export_size=2,
  export_device="auto",
  # export_hub_model_id="your_id/your_model",
)

json.dump(args, open("merge_llama3.1.json", "w", encoding="utf-8"), indent=2)

%cd /content/drive/MyDrive/5398/LLaMA-Factory

!llamafactory-cli export merge_llama3.1.json

In [ ]:
df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_system.json")
print(df_test.columns)

**1.2 Testing Finetuned Model**

In [ ]:
args = dict(
    model_name_or_path="llama3.1_lora_merged",
    template="llama3",
    finetuning_type="lora",
    max_new_tokens = 512
)
chat_model = ChatModel(args)

df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_system.json")
# column: 'instruction' -> model input, 'output' -> correct answer

rouge = Rouge()
results = []

for idx, row in df_test.iterrows():
    system_text = row["system"] if pd.notna(row["system"]) else ""

    user_content = row["instruction"]
    if pd.notna(row.get("input", "")) and row.get("input", "") != "":
      user_content += "\n" + row["input"]

    full_prompt = system_text + "\n" + user_content

    messages = [
        {"role": "user", "content": full_prompt}
    ]

    start_time = time.time()

    response = ""
    for new_text in chat_model.stream_chat(messages):
        response += new_text

    output = response

    inference_time = time.time() - start_time

    scores = rouge.get_scores(output, row["output"])[0]

    results.append({
        "prompt": row["instruction"],
        "system": row.get("system", ""),
        "reference": row["output"],
        "prediction": output,
        "rouge-1_f": scores['rouge-1']['f'],
        "rouge-2_f": scores['rouge-2']['f'],
        "rouge-l_f": scores['rouge-l']['f'],
        "inference_time_s": inference_time
    })

    torch_gc()


df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/MyDrive/5398/LLaMA-Factory/llama3.1_test_results.csv", index=False)

print("save llama3.1_test_results.csv")

**1.3 Testing Base Model**

In [ ]:
args = dict(
    model_name_or_path="meta-llama/Llama-3.1-8B",
    template="llama3",
    max_new_tokens=512
)
chat_model = ChatModel(args)

df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_system.json")


rouge = Rouge()
results = []

for idx, row in df_test.iterrows():
    system_text = row["system"] if pd.notna(row["system"]) else ""

    user_content = row["instruction"]
    if pd.notna(row.get("input", "")) and row.get("input", "") != "":
        user_content += "\n" + row["input"]

    full_prompt = system_text + "\n" + user_content

    messages = [
        {"role": "user", "content": full_prompt}
    ]

    start_time = time.time()

    response = ""
    for new_text in chat_model.stream_chat(messages):
        response += new_text

    output = response
    inference_time = time.time() - start_time

    scores = rouge.get_scores(output, row["output"])[0]

    results.append({
        "prompt": row["instruction"],
        "system": row.get("system", ""),
        "reference": row["output"],
        "prediction": output,
        "rouge-1_f": scores['rouge-1']['f'],
        "rouge-2_f": scores['rouge-2']['f'],
        "rouge-l_f": scores['rouge-l']['f'],
        "inference_time_s": inference_time
    })

    torch_gc()

df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/MyDrive/5398/LLaMA-Factory/llama3.1_base_test_results.csv", index=False)

print("save llama3.1_base_test_results.csv")

# **2. Deepseek R1**

**2.1 Fine Tune**

In [ ]:
args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
  dataset="df_train_llama",
  template="deepseekr1",
  finetuning_type="lora",
  lora_target="q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj",
  output_dir="DeepSeek-R1_lora",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=16,
  lr_scheduler_type="cosine",
  logging_steps=5,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_grad_norm=1.0,
  loraplus_lr_ratio=16.0,
  fp16=True,
  report_to="none",
)

json.dump(args, open("train_deepseekR1.json", "w", encoding="utf-8"), indent=2)

%cd /content/drive/MyDrive/5398/LLaMA-Factory
!llamafactory-cli train "/content/drive/MyDrive/5398/LLaMA-Factory/train_deepseekR1.json"

In [ ]:
args = dict(
  model_name_or_path="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
  adapter_name_or_path="DeepSeek-R1_lora",
  template="deepseekr1",
  finetuning_type="lora",
)
chat_model = ChatModel(args)

In [ ]:
messages = [
    {"role": "user", "content": '''You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:

[Positive Developments]:
1. ...

[Potential Concerns]:
1. ...

[Prediction & Analysis]
Prediction: ...
Analysis: ...

<</SYS>>

[Company Introduction]:

American Express Co is a leading entity in the Financial Services sector. Incorporated and publicly traded since 1977-05-18, the company has established its reputation as one of the key players in the market. As of today, American Express Co has a market capitalization of 168338.49 in USD, with 723.87 shares outstanding.

American Express Co operates primarily in the US, trading under the ticker AXP on the NEW YORK STOCK EXCHANGE, INC.. As a dominant force in the Financial Services space, the company continues to innovate and drive progress within the industry.

From 2023-05-07 to 2023-05-14, AXP's stock price decreased from 150.55 to 145.90. News during this period are listed below:

[Headline]: Credit cards: issuers could use their own flexible friends
[Summary]: Concerns profitability has peaked following a strong 2022 are not baseless, but even so credit cards remain a lucrative business

[Headline]: Dow's nearly 175-point fall led by losses in American Express, Nike shares
[Summary]: Dragged down by losses for shares of American Express and Nike, the Dow Jones Industrial Average is trading down Wednesday afternoon. Shares of American...

[Headline]: Chase is opening its first U.S. airport lounge in Boston — here's how you can get in
[Summary]: The new Chase Sapphire Lounge at Boston Airport features complimentary food, drinks, rest pods, massage chairs and more. Here's how you can get in.

[Headline]: American Express Co. stock outperforms competitors despite losses on the day
[Summary]: Shares of American Express Co. sank 0.04% to $147.93 Friday, on what proved to be an all-around dismal trading session for the stock market, with the S&P 500...

[Headline]: How To Build A Dividend Portfolio With $25,000 Among May's Top 30 Stocks
[Summary]: An investment strategy that combines dividend income with dividend growth, brings several benefits for dividend income investors. Click here to read more.

Some recent basic financials of AXP, reported at 2023-03-31, are presented below:

[Basic Financials]:

assetTurnoverTTM: 0.2614
bookValue: 25992
cashRatio: 0.2944926548987087
currentRatio: 0.7185
ebitPerShare: 2.9126
eps: 2.4409
ev: 246452.85
fcfPerShareTTM: 20.2611
grossMargin: 0.624
longtermDebtTotalAsset: 0.1744
longtermDebtTotalCapital: 0.2169
longtermDebtTotalEquity: 1.5827
netDebtToTotalCapital: 0.6532
netDebtToTotalEquity: 4.7667
netMargin: 0.1189
operatingMargin: 0.1419
payoutRatioTTM: 0.2247
pb: 4.7152
peTTM: 16.9489
pretaxMargin: 0.1419
psTTM: 2.1201
quickRatio: 0.7185
receivablesTurnoverTTM: 1.0476
roaTTM: 0.0327
roeTTM: 0.2955
roicTTM: 0.0412
rotcTTM: 0.0515
salesPerShare: 20.5309
sgaToSale: 0.376
totalDebtToEquity: 6.2969
totalDebtToTotalAsset: 0.694
totalDebtToTotalCapital: 0.863
totalRatio: 1.1239

Based on all the information before 2023-05-14, let's first analyze the positive developments and potential concerns for AXP. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. Then make your prediction of the AXP cryptocurrency price movement for next week (2023-05-14 to 2023-05-21). Provide a summary analysis to support your prediction.'''}
]

response = ""
for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text

torch_gc()

In [ ]:
args = dict(
  model_name_or_path="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
  adapter_name_or_path="DeepSeek-R1_lora",
  template="deepseekr1",
  finetuning_type="lora",
  export_dir="DeepseekR1_lora_merged",
  export_size=2,
  export_device="auto",
  # export_hub_model_id="your_id/your_model",
)

json.dump(args, open("merge_DeepseekR1.json", "w", encoding="utf-8"), indent=2)

%cd /content/drive/MyDrive/5398/LLaMA-Factory

!llamafactory-cli export merge_DeepseekR1.json

In [ ]:
df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_llama.json")
print(df_test.columns)

**2.2 Testing Finetuned Model**

In [ ]:
args = dict(
    model_name_or_path="DeepseekR1_lora_merged",
    template="deepseekr1",
    finetuning_type="lora"
)
chat_model = ChatModel(args)

df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_llama.json")


rouge = Rouge()
results = []


for idx, row in df_test.iterrows():
    messages = [{"role": "user", "content": row["instruction"]}]

    # 计时
    start_time = time.time()
    output = "".join(chat_model.stream_chat(messages))
    inference_time = time.time() - start_time

    scores = rouge.get_scores(output, row["output"])[0]

    results.append({
        "prompt": row["instruction"],
        "reference": row["output"],
        "prediction": output,
        "rouge-1_f": scores['rouge-1']['f'],
        "rouge-2_f": scores['rouge-2']['f'],
        "rouge-l_f": scores['rouge-l']['f'],
        "inference_time_s": inference_time
    })

    torch_gc()


df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/MyDrive/5398/LLaMA-Factory/test_results.csv", index=False)

print("save test_results.csv")

**2.3 Testing Base Model**

In [ ]:
args = dict(
    model_name_or_path="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    template="deepseekr1"
)
chat_model = ChatModel(args)

df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_llama.json")

rouge = Rouge()
results = []

for idx, row in df_test.iterrows():
    messages = [{"role": "user", "content": row["instruction"]}]

    # 计时
    start_time = time.time()
    output = "".join(chat_model.stream_chat(messages))
    inference_time = time.time() - start_time

    scores = rouge.get_scores(output, row["output"])[0]

    results.append({
        "prompt": row["instruction"],
        "reference": row["output"],
        "prediction": output,
        "rouge-1_f": scores['rouge-1']['f'],
        "rouge-2_f": scores['rouge-2']['f'],
        "rouge-l_f": scores['rouge-l']['f'],
        "inference_time_s": inference_time
    })

    torch_gc()

df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/MyDrive/5398/LLaMA-Factory/Deepseek_base_test_results.csv", index=False)

print("save Deepseek_base_test_results.csv")

# **3. Qwen 2.5 7b Instruct**

**3.1 Fine Tune**

In [ ]:
args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="Qwen/Qwen2.5-7B-Instruct",
  dataset="df_train_system",
  template="qwen",
  finetuning_type="lora",
  lora_target="q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj",
  output_dir="Qwen2.5_lora",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=16,
  lr_scheduler_type="cosine",
  logging_steps=5,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_grad_norm=1.0,
  loraplus_lr_ratio=16.0,
  fp16=True,
  report_to="none",
)

json.dump(args, open("train_Qwen2.5.json", "w", encoding="utf-8"), indent=2)

%cd /content/drive/MyDrive/5398/LLaMA-Factory
!llamafactory-cli train "/content/drive/MyDrive/5398/LLaMA-Factory/train_Qwen2.5.json"

In [ ]:
args = dict(
  model_name_or_path="Qwen/Qwen2.5-7B-Instruct",
  adapter_name_or_path="Qwen2.5_lora",
  template="qwen",
  finetuning_type="lora"
)
chat_model = ChatModel(args)


In [ ]:
system_text = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ...\n\n"
messages = [
    {"role": "user", "content": system_text + "\n" +"[Company Introduction]:\n\nAmerican Express Co is a leading entity in the Financial Services sector. Incorporated and publicly traded since 1977-05-18, the company has established its reputation as one of the key players in the market. As of today, American Express Co has a market capitalization of 168338.49 in USD, with 723.87 shares outstanding.\n\nAmerican Express Co operates primarily in the US, trading under the ticker AXP on the NEW YORK STOCK EXCHANGE, INC.. As a dominant force in the Financial Services space, the company continues to innovate and drive progress within the industry.\n\nFrom 2024-02-11 to 2024-02-18, AXP's stock price increased from 211.81 to 211.90. News during this period are listed below:\n\n[Headline]: American Express: Buy, Sell, or Hold?\n[Summary]: With shares at all-time highs, investors still won't struggle in finding reasons to be bullish.\n\n[Headline]: American Express Co. stock underperforms Monday when compared to competitors\n[Summary]: Shares of American Express Co. slipped 0.10% to $212.26 Monday, on what proved to be an all-around mixed trading session for the stock market, with the Dow...\n\n[Headline]: The Next Berkshire Hathaway? 3 Blue-Chip Stocks That Investors Shouldn’t Ignore.\n[Summary]: Blue chip stocks are what many investors look for when trying to add new stocks to their portfolios. What really are these companies? Well, a blue chip company is generally a well-known and well-established company that has a history of generating consistent profits. They have grown their revenue in the past and have worked to make a successful company. That’s why these are usually more reliable to invest in since they are not small startup companies that just hit the market and carry tons of ri\n\n[Headline]: Tracking Mario Gabelli's Gabelli Funds 13F Portfolio - Q4 2023 Update\n[Summary]: Gabelli Funds' 13F portfolio value increased by approximately 5% in Q4 2023. Click here to read more about Mario Gabelli's holdings and trades for Q4 2023.\n\n[Headline]: American Express Co. stock underperforms Thursday when compared to competitors despite daily gains\n[Summary]: Shares of American Express Co. inched 0.77% higher to $212.53 Thursday, on what proved to be an all-around positive trading session for the stock market,...\n\nFrom 2024-02-18 to 2024-02-25, AXP's stock price increased from 211.90 to 213.90. News during this period are listed below:\n\n[Headline]: The Dow is due for a change. These sectors may be affected.\n[Summary]: The keepers of the Dow Jones Industrial Average will have to make at least one change to the venerable index next week, and there’s a very good chance...\n\n[Headline]: Discover Stock Jumps on Capital One Deal News\n[Summary]: Shares of Discover Financial Services jumped in early trading following the news that Capital One will buy the credit-card company for more than $35 billion.  Discover's shares were up 14% in recent trading, while Capital One's rose 1%.  Capital One stock was up more than 4%.\n\n[Headline]: Got $1,000? 3 Payments Stocks to Buy and Hold Forever\n[Summary]: Americans spend trillions of dollars on debit and credit cards; these three companies are behind almost every transaction.\n\n[Headline]: Are You a Growth Investor? This 1 Stock Could Be the Perfect Pick\n[Summary]: The Zacks Style Scores offers investors a way to easily find top-rated stocks based on their investing style. Here's why you should take advantage.\n\n[Headline]: Nearly One-Fifth of Warren Buffett-Led Berkshire Hathaway's $368 Billion Stock Portfolio Is Invested in These 2 Financial Giants\n[Summary]: These two industry heavyweights should be on your investing radar.\n\nSome recent basic financials of AXP, reported at 2023-12-31, are presented below:\n\n[Basic Financials]:\n\nassetTurnoverTTM: 0.2635\nbookValue: 28057\ncashRatio: 0.31986958884260097\ncurrentRatio: 0.7395\nebitPerShare: 3.4553\neps: 2.6589\nev: 268910.19\nfcfMargin: 0.3691\nfcfPerShareTTM: 23.5076\ngrossMargin: 0.6334\nlongtermDebtTotalAsset: 0.1833\nlongtermDebtTotalCapital: 0.232\nlongtermDebtTotalEquity: 1.706\nnetDebtToTotalCapital: 0.6415\nnetDebtToTotalEquity: 4.7185\nnetMargin: 0.1125\noperatingMargin: 0.1462\npayoutRatioTTM: 0.2126\npb: 4.8659\npeTTM: 16.3032\npfcfTTM: 7.6988\npretaxMargin: 0.1462\npsTTM: 2.0881\nquickRatio: 0.7395\nreceivablesTurnoverTTM: 1.1117\nroaTTM: 0.0338\nroeTTM: 0.3099\nroicTTM: 0.0422\nrotcTTM: 0.053\nsalesPerShare: 23.6369\nsgaToSale: 0.3666\ntotalDebtToEquity: 6.355\ntotalDebtToTotalAsset: 0.6829\ntotalDebtToTotalCapital: 0.864\ntotalRatio: 1.1204\n\nBased on all the information before 2024-02-25, let's first analyze the positive developments and potential concerns for AXP. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. Then make your prediction of the AXP cryptocurrency price movement for next week (2024-02-25 to 2024-03-03). Provide a summary analysis to support your prediction."}
]

response = ""
for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text

torch_gc()

In [ ]:
args = dict(
  model_name_or_path="Qwen/Qwen2.5-7B-Instruct",
  adapter_name_or_path="Qwen2.5_lora",
  template="qwen",
  finetuning_type="lora",
  export_dir="Qwen2.5_lora_merged",
  export_size=2,
  export_device="auto",
  # export_hub_model_id="your_id/your_model",

json.dump(args, open("merge_Qwen2.5.json", "w", encoding="utf-8"), indent=2)

%cd /content/drive/MyDrive/5398/LLaMA-Factory

!llamafactory-cli export merge_Qwen2.5.json

In [ ]:
df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_system.json")
print(df_test.columns)

**3.2 Testing Finetuned Model**

In [ ]:
args = dict(
    model_name_or_path="Qwen2.5_lora_merged",
    template="qwen",
    finetuning_type="lora",
)
chat_model = ChatModel(args)

df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_system.json")

rouge = Rouge()
results = []

for idx, row in df_test.iterrows():
    system_text = row["system"] if pd.notna(row["system"]) else ""

    user_content = row["instruction"]
    if pd.notna(row.get("input", "")) and row.get("input", "") != "":
      user_content += "\n" + row["input"]

    full_prompt = system_text + "\n" + user_content

    messages = [
        {"role": "user", "content": full_prompt}
    ]

    start_time = time.time()

    response = ""
    for new_text in chat_model.stream_chat(messages):
        response += new_text

    output = response

    inference_time = time.time() - start_time

    scores = rouge.get_scores(output, row["output"])[0]

    results.append({
        "prompt": row["instruction"],
        "system": row.get("system", ""),
        "reference": row["output"],
        "prediction": output,
        "rouge-1_f": scores['rouge-1']['f'],
        "rouge-2_f": scores['rouge-2']['f'],
        "rouge-l_f": scores['rouge-l']['f'],
        "inference_time_s": inference_time
    })

    torch_gc()

df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/MyDrive/5398/LLaMA-Factory/Qwen2.5_test_results.csv", index=False)

print("save Qwen2.5_test_results.csv")


**3.3 Testing base model**

In [ ]:
args = dict(
    model_name_or_path="Qwen/Qwen2.5-7B-Instruct",
    template="qwen",
)
chat_model = ChatModel(args)

df_test = pd.read_json("/content/drive/MyDrive/5398/LLaMA-Factory/data/df_test_system.json")


rouge = Rouge()
results = []


for idx, row in df_test.iterrows():
    system_text = row["system"] if pd.notna(row["system"]) else ""

    user_content = row["instruction"]
    if pd.notna(row.get("input", "")) and row.get("input", "") != "":
        user_content += "\n" + row["input"]

    full_prompt = system_text + "\n" + user_content

    messages = [
        {"role": "user", "content": full_prompt}
    ]

    start_time = time.time()

    response = ""
    for new_text in chat_model.stream_chat(messages):
        response += new_text

    output = response
    inference_time = time.time() - start_time

    scores = rouge.get_scores(output, row["output"])[0]

    results.append({
        "prompt": row["instruction"],
        "system": row.get("system", ""),
        "reference": row["output"],
        "prediction": output,
        "rouge-1_f": scores['rouge-1']['f'],
        "rouge-2_f": scores['rouge-2']['f'],
        "rouge-l_f": scores['rouge-l']['f'],
        "inference_time_s": inference_time
    })

    torch_gc()

df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/MyDrive/5398/LLaMA-Factory/Qwen2.5_base_test_results.csv", index=False)

print("save Qwen2.5_base_test_results.csv")